In [1]:
import os
import shutil
import pandas as pd

In [2]:
train_df = pd.read_csv('./dataset/train_df.csv')
train_df.head()

,index,file_name,class,state,label
0,0,10000.png,transistor,good,transistor-good
1,1,10001.png,capsule,good,capsule-good
2,2,10002.png,transistor,good,transistor-good
3,3,10003.png,wood,good,wood-good
4,4,10004.png,bottle,good,bottle-good


In [3]:
train_df.tail()

,index,file_name,class,state,label
4272,4272,14272.png,transistor,good,transistor-good
4273,4273,14273.png,transistor,good,transistor-good
4274,4274,14274.png,grid,good,grid-good
4275,4275,14275.png,zipper,good,zipper-good
4276,4276,14276.png,screw,good,screw-good


In [4]:
from tensorflow import keras

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
data_pre = train_df["file_name"].to_numpy()
target = train_df["label"].to_numpy()
train_x, valid_x, train_y, valid_y = train_test_split(
    data_pre, target, test_size=0.2, stratify=target
)
print(train_x.shape,valid_x.shape)

(3421,) (856,)


In [8]:
train_path = "./dataset/gen_directory/train/"
valid_path = "./dataset/gen_directory/valid/"

if not os.path.exists("./dataset/gen_directory/"):
        os.mkdir("./dataset/gen_directory/")

if not os.path.exists(train_path):
        os.mkdir(train_path)
if not os.path.exists(valid_path):
        os.mkdir(valid_path)

for label in train_df["label"].unique():
    if not os.path.exists(train_path+label):
        os.mkdir(train_path+label)
    if not os.path.exists(valid_path+label):
        os.mkdir(valid_path+label)       

In [9]:
for file, label in zip(train_x,train_y):
    src = "./dataset/train/"+file
    dst = train_path + label + '/' + label + '_train_' + file
    shutil.copy(src, dst)

In [ ]:
for file, label in zip(valid_x,valid_y):
    src = "./dataset/train/"+file
    dst = valid_path + label + '/' + label + '_train_' + file
    shutil.copy(src, dst)

In [10]:
INPUT_SHAPE = (700,700,3)
BATCH_SIZE = 32

In [28]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    vertical_flip=True,
    fill_mode='reflect'
)
train_generator = train_datagen.flow_from_directory(
    directory= train_path,
    target_size= INPUT_SHAPE[:2],
    save_to_dir='./gen_train/',
    batch_size=BATCH_SIZE,
    class_mode = "categorical"
)

Found 4099 images belonging to 88 classes.


In [29]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(    rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
    directory= valid_path,
    target_size= INPUT_SHAPE[:2],
    save_to_dir='./gen_valid/',
    batch_size=BATCH_SIZE,
    class_mode = "categorical"
)

Found 1534 images belonging to 88 classes.


In [30]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [36]:
test_generator = test_datagen.flow_from_directory(
    directory='./dataset/test',
    save_to_dir='./gen_test/',
    target_size= INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode= None,
)

Found 2154 images belonging to 1 classes.


In [37]:
transfer_model = keras.applications.Xception(weights='imagenet',
                                         include_top = False,
                                         input_shape = INPUT_SHAPE)
transfer_model.trainable = False
# transfer_model.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 700, 700, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 349, 349, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 349, 349, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

 2D)                                                                                              
                                                                                                  
 block4_sepconv1_bn (BatchNorma  (None, 87, 87, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 87, 87, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 87, 87, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
          

                                                                                                  
 block7_sepconv1_act (Activatio  (None, 44, 44, 728)  0          ['add_40[0][0]']                 
 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 44, 44, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 44, 44, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_se

 2D)                                                                                              
                                                                                                  
 block9_sepconv3_bn (BatchNorma  (None, 44, 44, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_43 (Add)                   (None, 44, 44, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_42[0][0]']                 
                                                                                                  
 block10_sepconv1_act (Activati  (None, 44, 44, 728)  0          ['add_43[0][0]']                 
 on)                                                                                              
          

 block12_sepconv2_bn (BatchNorm  (None, 44, 44, 728)  2912       ['block12_sepconv2[0][0]']       
 alization)                                                                                       
                                                                                                  
 block12_sepconv3_act (Activati  (None, 44, 44, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 44, 44, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 44, 44, 728)  2912       ['block12_sepconv3[0][0]']       
 alization

In [38]:
finetune_model = keras.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(keras.layers.Flatten())
finetune_model.add(keras.layers.Dense(200, activation='relu'))
finetune_model.add(keras.layers.Dense(88, activation='softmax'))

finetune_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 22, 22, 2048)      20861480  
                                                                 
 flatten_3 (Flatten)         (None, 991232)            0         
                                                                 
 dense_6 (Dense)             (None, 200)               198246600 
                                                                 
 dense_7 (Dense)             (None, 88)                17688     
                                                                 
Total params: 219,125,768
Trainable params: 198,264,288
Non-trainable params: 20,861,480
_________________________________________________________________


In [39]:
finetune_model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=0.0002),
                       metrics='accuracy')                    

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
hist = finetune_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

C:\Users\Mu-jun\AppData\Local\Temp/ipykernel_3536/204086450.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = finetune_model.fit_generator(generator=train_generator,


Epoch 1/10
1024/1024 [==============================] - 5964s 6s/step - loss: 2.4626 - accuracy: 0.7253 - val_loss: 1.5445 - val_accuracy: 0.7565
Epoch 2/10
 428/1024 [===========>..................] - ETA: 43:19 - loss: 1.6285 - accuracy: 0.7880

In [ ]:
plt.plot(hist.history['loss'], c='r')
plt.plot(hist.history['val_loss'], c='b')

plt.show()